<a href="https://colab.research.google.com/github/danielbryksil/Calculadora-LF-Secund-rio/blob/main/PU_LF_CDI_com_Fluxo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PU % CDI com Fluxo


In [ ]:
!pip install python-bcb
!pip install pandas_market_calendars

In [ ]:
from datetime import datetime
import pandas as pd
from bcb import sgs
import numpy as np
from datetime import datetime, timedelta

In [ ]:
dados_excel = pd.read_excel('/content/feriados.xlsx')
feriados = dados_excel['2001-01-01'].tolist()

In [ ]:
df_eventos = pd.read_excel('/content/data.xlsx')
df_eventos

,data,amortização
0,2024-04-09,0
1,2024-10-09,0
2,2025-04-09,0
3,2025-10-09,0
4,2026-04-09,0
5,2026-10-09,0
6,2027-04-09,500000


In [ ]:
data_operação = input("Qual a data da operação (aaaa-mm-dd)? ")

Qual a data da operação (aaaa-mm-dd)? 2024-02-20


In [ ]:
data_str = data_operação

# Converta a string para um objeto datetime
data = datetime.strptime(data_str, '%Y-%m-%d')

# Subtraia um dia usando timedelta
data_menos_um_dia = data - timedelta(days=1)

# Converta o resultado de volta para o formato desejado
data_menos_um_dia_operação = data_menos_um_dia.strftime('%Y-%m-%d')
data_menos_um_dia_operação

'2024-02-19'

In [ ]:
data_ultimo_pagamento = input("Qual a data do último pagamento de juros (aaaa-mm-dd)? ")

Qual a data do último pagamento de juros (aaaa-mm-dd)? 2023-10-09


In [ ]:
PU_inicial = float(input("Qual o PU inicial? "))

Qual o PU inicial? 500000


In [ ]:
taxa_emissão = float(input("Qual a taxa de emissão?"))

taxa_emissão = taxa_emissão/100

Qual a taxa de emissão?118


In [ ]:
tx_neg = float(input("Qual a taxa de negociação?"))
tx_neg = tx_neg /100

Qual a taxa de negociação?112


In [ ]:
df_eventos['dias'] = (df_eventos['data'] - datetime.strptime(data_operação, '%Y-%m-%d')).dt.days

In [ ]:
data_cdi = sgs.get(('CDI', 12), last=3000)
data_cdi = data_cdi['CDI']
data_cdi

Date
2012-03-07    0.038622
2012-03-08    0.036129
2012-03-09    0.036056
2012-03-12    0.035948
2012-03-13    0.035875
                ...   
2024-02-09    0.041957
2024-02-14    0.041957
2024-02-15    0.041957
2024-02-16    0.041957
2024-02-19    0.041957
Name: CDI, Length: 3000, dtype: float64

In [ ]:
data_filt = data_cdi.loc[data_ultimo_pagamento:data_menos_um_dia_operação]
data_filt

Date
2023-10-09    0.047279
2023-10-10    0.047279
2023-10-11    0.047279
2023-10-13    0.047279
2023-10-16    0.047279
                ...   
2024-02-09    0.041957
2024-02-14    0.041957
2024-02-15    0.041957
2024-02-16    0.041957
2024-02-19    0.041957
Name: CDI, Length: 89, dtype: float64

In [ ]:
import pandas as pd
import requests

# URL da página
url = 'https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/TxRef1.asp'

# Realiza a solicitação HTTP
response = requests.get(url)

# Verifica se a solicitação foi bem-sucedida (código de status 200)
if response.status_code == 200:
    # Lê a tabela HTML diretamente usando o pandas
    dfs = pd.read_html(response.text, flavor='bs4')  # 'bs4' indica o uso do BeautifulSoup para análise HTML

    # O resultado será uma lista de DataFrames, onde cada DataFrame representa uma tabela na página
    # Dependendo da página, você pode precisar procurar qual DataFrame contém os dados desejados
    for i, df in enumerate(dfs):
        print(f"DataFrame {i + 1}:\n{df}")
else:
    print(f"Falha na solicitação. Código de status: {response.status_code}")

<ipython-input-14-14070766bd13>:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(response.text, flavor='bs4')  # 'bs4' indica o uso do BeautifulSoup para análise HTML


DataFrame 1:
                                                   0            1
0                          Atualizado em: 19/02/2024  Retroativo:
1  Taxa: Ajuste cupom  Alumínio  DI x Anbid  Anbi...          NaN
DataFrame 2:
                                                     0          1         2
0    Dias Corridos DI x pré 252(2)(4) 360(1) 1 11,1...        NaN       NaN
1                                        Dias Corridos   DI x pré  DI x pré
2                                        Dias Corridos  252(2)(4)    360(1)
3                                                    1       1115       000
4                                                    7       1115      1139
..                                                 ...        ...       ...
258                                               9493       1082      1060
259                                               9674       1082      1060
260                                              10801       1082      1060
261             

In [ ]:
di_tabela = dfs[2]
di_tabela = di_tabela.set_index(0)
di_tabela = di_tabela.iloc[2:]
di_tabela = di_tabela[1]
di_tabela = di_tabela.astype(float)
di_tabela.index = di_tabela.index.astype(int)
di_tabela = di_tabela/100
di_tabela

0
1        11.15
7        11.15
9        11.15
10       11.15
11       11.15
         ...  
9493     10.82
9674     10.82
10801    10.82
11410    10.82
13328    10.82
Name: 1, Length: 260, dtype: float64

In [ ]:
di_dia_list = []

for a in df_eventos['dias']:
    indice_mais_proximo = (di_tabela.index.to_series() - a).abs().idxmin()
    tx_di = di_tabela.loc[indice_mais_proximo]
    tx_di = tx_di/100
    di_dia_list.append(tx_di)

df_eventos['di_dia'] = di_dia_list


In [ ]:
num_dias_uteis = []

for a in df_eventos['data']:
    datas = pd.date_range(start=data_operação, end=a, freq='B')
    dias_uteis = datas[~datas.isin(feriados)]
    dias_uteis_b = len(dias_uteis)-1
    num_dias_uteis.append(dias_uteis_b)


df_eventos['num_dias_uteis'] = num_dias_uteis


In [ ]:
lista1 = df_eventos['num_dias_uteis']
lista1 = pd.concat([pd.Series([0]), lista1], ignore_index=True)
df_eventos['num_dias_uteis_apoio'] = lista1[:-1]
lista2 = df_eventos['di_dia']
lista2 = pd.concat([pd.Series([0]), lista2], ignore_index=True)
df_eventos['di_dia_apoio'] = lista2[:-1]

In [ ]:
df_eventos['fator'] = ((((1+df_eventos['di_dia'])**(1/252))-1)*taxa_emissão+1)**df_eventos['num_dias_uteis']/((((1+df_eventos['di_dia_apoio'])**(1/252))-1)*taxa_emissão+1)**df_eventos['num_dias_uteis_apoio']
df_eventos['cupom'] = PU_inicial*(df_eventos['fator']-1)

In [ ]:
di = (data_filt/100 * taxa_emissão)+1
di_per = di.product()
di_per

1.0481086196496978

In [ ]:
pu_par = di_per*PU_inicial
pu_par

524054.3098248489

In [ ]:
df_eventos.loc[0,'cupom'] = pu_par*((df_eventos.loc[0,'fator'])-1)+(pu_par-PU_inicial)
df_eventos['desconto'] = ((((1+df_eventos['di_dia'])**(1/252))-1)*(tx_neg)+1)**df_eventos['num_dias_uteis']
df_eventos['pu'] = (df_eventos['cupom']+df_eventos['amortização'])/df_eventos['desconto']


In [ ]:
pu_final = df_eventos['pu'].sum()
pu_final

531930.097531073

In [ ]:
quantidade = float(input('qual a quantidade?'))

qual a quantidade?7


In [ ]:
financeiro = round(pu_final*quantidade,2)
financeiro

3723510.68

In [ ]:
import locale

# Configuração da localidade para o formato financeiro
locale.setlocale(locale.LC_ALL, '')

# Valor em formato numpy.float64
valor_float64 = np.float64(financeiro)

# Converte para uma string no formato financeiro
valor_formatado = locale.currency(valor_float64, grouping=True)

valor_formatado

'$3,723,510.68'